## Programmation vectorielle

Le but des exercices est

* d'avoir un programme qui donne la bonne réponse
* qui soit le plus rapide possible (et pour cela on utilise massivement Numpy)

En règle général si vous avez des `for` imbriqués c'est mauvais signe.

In [1]:
import numpy as np

np.set_printoptions(precision=10, linewidth=150, suppress=True)

## Méthode du pivot de Gauss partiel

L'ennoncé est dans le cours (et lire le cours peut servir). On verifiera sur le cas qui génère des
erreurs d'arrondis.

La structure de la matrice A après 3 itérations ressemble à cela :

```
 ----------------
  \              |
    \            |
      \          |
       |         |
   0   |         |
       |         |
       -----------
```
Aussi il est important que ne pas détruire le travail déjà fait à savoir de garder les 0 sous la diagonale.
Cela implique qu'on ne doit pas changer la 4e ligne avec une ligne au dessus. On ne regarde donc que les
valeurs de la 4e colonne qui sont en dessous de la diagonale pour choisir le nouveau pivot.

In [2]:
def solve_gauss_partial(A, b):   # on prend le max dans la colonne i parmi les lignes en dessous (plus facile)
    for i in range(len(A)-1):
        pivot = i + np.argmax(np.abs(A[i:, i]))  # il n'y a que 3 lignes à ajouter pour échanger les lignes
        A[[i, pivot]] = A[[pivot, i]]
        b[[i, pivot]] = b[[pivot, i]]
        # la suite est comme le pivot de Gauss normal
        coefs = - A[i+1:,i] / A[i,i]
        A[i+1:, i:] += np.outer(coefs, A[i, i:]) # ou np.einsum('i,j -> ij', coefs, A[i, i:])
        b[i+1:] += coefs * b[i]                  # multiplication terme à terme
    # A est maintenant triangulaire surpérieur
    res = np.zeros(len(b), dtype=b.dtype)
    res[-1] = b[-1] / A[-1,-1]
    for i in range(len(A)-1)[::-1]:
        res[i] = (b[i] - A[i,i+1:] @ res[i+1:]) / A[i,i]
    return res

In [3]:
e = 1E-6
A = np.array([[e, 1], [1, 2]], dtype='float32')
b = np.array([1., 3.], dtype='float32')
print(f"A\n {A} \nb\n {b}\n")
x = solve_gauss_partial(A, b)
print('solution : ',x)
print('vérification\n', A@x)

A
 [[0.000001 1.      ]
 [1.       2.      ]] 
b
 [1. 3.]

solution :  [1.0000019  0.99999905]
vérification
 [3.       0.999997]


## Factorisation de Choleski

Il s'agit de décomposer A en $A = B\, B^T$ avec B une matrice triangulaire inférieure. Cela n'est possible
que si la matrice A est symétrique et définie positive (c'est d'ailleurs une facon de vérifier qu'une
matrice est définie positive).

Écrire l'algorithme de Choleski qui prend A et retourne B (pour deviner l'algorithme, essayez de trouver les 
coefficients de B à partir des coefficients de A sur une matrice A 4x4).

**Calculs** A faire sur papier, c'est bien plus simple !

$$
A = B \, B^T = 
\begin{bmatrix}
b_{11} \; 0 \; \; \ldots 0 \\
b_{21} \; b_{22} \ldots 0 \\
 \vdots \\
b_{n1} b_{n2} \ldots b_{n,n} \\
\end{bmatrix}
\;
\begin{bmatrix}
b_{11} \; b_{21} \; \; \ldots b_{n1} \\
0 \; \; b_{22} \ldots \; b_{n2} \\
 \vdots \\
0 \; \; 0 \; \; \ldots \; b_{n,n} \\
\end{bmatrix}
=
\begin{bmatrix}
b_{11}^2 \quad b_{11} b_{21} \quad \ldots \quad b_{11} b_{n1} \\
x \; \; \sum_{i=1}^2 b_{2i}^2 \; \ldots \; \sum_{i=1}^2 b_{2i} b_{ni} \\
 \vdots \\
x \quad x \quad \ldots \qquad \sum_{i=1}^n b_{n,i}^2 \\
\end{bmatrix}
$$

avec $x$ la même valeur que de l'autre coté de la diagonale (je vous laisse vérifier et il le faut puisque A
est symétrique).

On voit tout de suite que $b_{11} = \sqrt{a_{11}}$ et maintenant qu'on a $b_{11}$ on peut trouver 
toute la première ligne de $B^T$ : $b_{j1} = a_{1j} / b_{11}$. 

Une fois qu'on connait la première ligne de $B^T$, on s'attaque à la deuxième en commencant par trouver
$b_{22}$ puis ensuite tous les autres éléments de la ligne comme on a fait pour la première ligne.

On a donc dans le cas général :

* $b_{ii} = \sqrt{a_{ii} -  \sum_{k=1}^{i-1} b_{ik}^2} $
* $\displaystyle b_{ji} = \frac{a_{ij} - \sum_{k=1}^{i-1} b_{ik} \, b_{jk}}{b_{ii}} = \frac{a_{ij} - \sum_{k=1}^{i-1} b_{ik} \, b^T_{kj}}{b_{ii}} \quad \forall j > i$

In [4]:
def Choleski(A):
    B = np.zeros(A.shape)
    for i in range(len(A)):
        B[i,i] = np.sqrt(A[i,i] - np.sum(np.square(B[i, :i])))         # garanti ok car A est def positive
        B[i+1:, i] = (A[i, i+1:] - B[i, :i] @ B.T[:i, i+1:]) / B[i,i]  # les ∑ sous forme de prod. scalaire
    return B

Rappel : pas de boucles `for` imbriquées mais des opérations vectorielles et matricielles (sur des sous-matrices).

Créer une matrice symétrique définie positive et vérifier que son programme marche bien.

In [5]:
A = np.random.randint(10, size=(4,4))
A = A + A.T                             # symmétrique
A = A + np.diag(A.sum(axis=0))          # diagonale dominante
print('A:\n', A)
B = Choleski(A)
print('B\n', B)
print('vérification\n', B @ B.T)

A:
 [[40  2  6 16]
 [ 2 22  4  4]
 [ 6  4 41  7]
 [16  4  7 63]]
B
 [[6.3245553203 0.           0.           0.          ]
 [0.316227766  4.6797435827 0.           0.          ]
 [0.9486832981 0.7906416099 6.2829042524 0.          ]
 [2.5298221281 0.6837981491 0.6460962268 7.4642467642]]
vérification
 [[40.  2.  6. 16.]
 [ 2. 22.  4.  4.]
 [ 6.  4. 41.  7.]
 [16.  4.  7. 63.]]


## Amméliorer Jacobi

Lorsqu'on écrit une itération de la méthode de Jacobi avec l'ensemble des coefficients, on constate que
si on calcule la nouvelle valeur de **x** élément par élement alors lorsqu'on veut mettre à jour `x[1]`, 
on connait déjà `x[0]`. Idem lorsqu'on met à jour `x[2]` on connait déjà `x[0]` et `x[1]`, etc.

L'idée de la version amméliorée de Jacobi est d'utiliser la nouvelle valeur de `x[0]` et non pas l'ancienne
comme c'est le cas dans l'algorithme du cours. Ainsi en utilisant des valeurs mise à jour on peut espérer
converger plus vite.

Dans ce chaque itération demande un calcul ligne par ligne et donc une boucle `for` dans la boucle `for` sur
les itérations.

#### Test d'arrêt

On ajoutera un argument `error` à la fonction qui indique la précision désirée du résultat. Par
défaut sa valeur est de `1E-6` et pour offrir une bonne garantie on arrête l'algorithme lorsque
$||x_{t+1} - x_t|| < \texttt{error}\, / \, 10$.

In [11]:
def Jacobi(A, b, error=1E-6, verbose=False):  # version avec double boucle (for dans le while des itérations)
    M = np.diag(A)        # attention, c'est un vecteur
    N = np.diag(M) - A    # np.diag d'une matrice donne un vecteur, np.diag d'un vecteur donne une matrice
    x = np.random.random(len(b))
    previous_x = np.zeros(len(b))
    err = (error / 10) ** 2
    while np.sum(np.square(x - previous_x))  > err:
        previous_x = x.copy()
        if verbose:
            print(f"x = {x}")
        x = (N @ x + b) / M
        #for i in range(len(x)):
        #    x[i] = (N[i] @ x + b[i]) / M[i]   # on utilise à droite le x modifié (au lieu de previous_x)
    return x

In [12]:
A = np.random.randint(10, size=(4,4))
A = A + np.diag(A.sum(axis=0))
b = A.sum(axis=1)                     # ainsi la solution est [1,1,1,1]
print('A:\n', A, "\nb:\n", b, "\n")

Jacobi(A,b, verbose=True)

A:
 [[30  1  5  3]
 [ 5 16  8  7]
 [ 3  2 29  1]
 [ 4  1  6 11]] 
b:
 [39 36 35 22] 

x = [0.5280539342 0.6839821315 0.4162214528 0.5922301959]
x = [1.1486073339 1.6177717085 1.0846773703 1.5187703104]
x = [0.9134176836 0.6842595122 0.9241332507 0.8436122486]
x = [1.0388079163 1.1334099898 1.0361246785 1.1015700226]
x = [0.9793752183 0.925373302  0.9832822844 0.9540554795]
x = [1.0098682946 1.0349048298 1.0088645607 1.0234029198]
x = [0.9950187869 0.9822451002 0.9957649151 0.988403148 ]
x = [1.0024573627 1.0087477943 1.0021396652 1.0057354783]
x = [0.9987782482 0.9956529698 0.9989447188 0.9971440694]
x = [1.0006063743 1.0021589077 1.0005246637 1.0014150659]
x = [0.9996990859 0.9989290849 0.9997395861 0.9992970557]
x = [1.0001493939 1.0005317807 1.0001292247 1.0003488232]
x = [0.9999258542 0.9997360919 0.9999358425 0.999826845 ]
x = [1.0000368053 1.0001310046 1.0000318417 1.0000859487]
x = [0.9999817314 0.9999349749 0.999984194  0.9999573385]
x = [1.000009068  1.0000322763 1.0000078454 

array([0.9999999959, 0.9999999855, 0.9999999965, 0.9999999905])

La méthode de base de Jacobi sur le même système matriciel prend nettement plus d'itérations :

In [8]:
M = np.diag(A)        # attention, c'est un vecteur
N = np.diag(M) - A    # np.diag d'une matrice donne un vecteur, np.diag d'un vecteur donne une matrice
print(f"M:\n {np.diag(M)}\nN:\n {N}\n")

x = np.random.random(4)
previous_x = np.zeros(len(b))
err = (1E-6 / 10) ** 2
while np.sum(np.square(x - previous_x))  > err:
    previous_x = x.copy()
    print(f"x = {x}")
    x = (N @ x + b) / M

M:
 [[18  0  0  0]
 [ 0 11  0  0]
 [ 0  0 36  0]
 [ 0  0  0 28]]
N:
 [[ 0 -5 -8 -7]
 [ 0  0 -5 -4]
 [-1 -4  0 -5]
 [-1  0 -9  0]]

x = [0.662673753  0.0370129212 0.4983000827 0.8444163963]
x = [1.5509788866 1.2846212729 1.1379775717 1.1733080537]
x = [0.7922175937 0.8742618115 0.928999882  0.9359722489]
x = [1.0913825636 1.0555555995 1.0286353866 1.0302422667]
x = [0.9600801691 0.9759867273 0.9870884362 0.9875321056]
x = [1.0172574519 1.0104026724 1.0055086776 1.0055758537]
x = [0.9924936801 0.9954684725 0.9975903497 0.9976130161]
x = [1.0032579849 1.0019632897 1.0010435375 1.0010426133]
x = [0.9985853866 0.9991465327 0.9995465497 0.9995482206]
x = [1.0006142997 1.0003703972 1.0001968717 1.0001962738]
x = [0.9997332847 0.9998391406 0.9999145206 0.9999147805]
x = [1.0001158149 1.0000698432 1.0000371181 1.0000370011]
x = [0.9999497129 0.9999696732 0.9999838835 0.9999839329]
x = [1.0000218353 1.0000131682 1.000006998  1.0000069763]
x = [0.9999905189 0.9999942823 0.9999969614 0.9999969708]

## Bonus

Avoir une boucle qui travaille sur les indices n'est pas satisfaisant. Essayons de comprendre ce qui se passe lorsqu'on utilise la valeur modifiée de `x` dans la boucle `for`. À un instant donnée, lorsque `i == 3` par exemple, les valeurs précédentes, `x[0], x[1], x[2]`, ont déjà été mises à jour. Par contre les valeurs qui suivent, `x[4], x[5]...` sont toujours les anciennes. Donc les valeurs à gauche de la diagonale sont les nouvelles quand celles à droite de la diagonale sont les anciennces.

Cela revient à découper la matrice **A** en une matrice triangulaire inférieure **L** - une matrice triangulaire supérieure strictement **U**. Maintenant la méthode itérative est

$$
{\bf L} \, {\bf x}^{k+1} = {\bf U} \, {\bf x}^k + {\bf b}
$$

Inverser **L** est compliqué mais résoudre ${\bf L} \, {\bf x} = {\bf y}$ est rapide comme on l'a déjà vu. Alors faisons cela :

In [9]:
def Jacobi_bonus(A, b, error=1E-6, verbose=False):
    L = np.tril(A) 
    U = -np.triu(A, k=1) 
    if verbose:
        print(f"L:\n {L}\nU\n {U}\n")
    previous_x = np.zeros(len(b))
    x = np.random.random(len(b))
    err = (error / 10) ** 2
    while np.sum(np.square(x - previous_x))  > err:
        previous_x = x.copy()
        if verbose:
            print(f"x = {x}")
        # on résoud  L x = U x + b  avec L matrice triangulaire inférieure
        y = U @ x + b
        x[0] = y[0] / L[0,0]
        for i in range(1,len(L)):
            x[i] = (y[i] - L[i,:i] @ x[:i]) / L[i,i]
    return x

In [10]:
print('A:\n', A, "\nb:\n", b, "\n")

Jacobi_bonus(A,b, verbose=True)

A:
 [[18  5  8  7]
 [ 0 11  5  4]
 [ 1  4 36  5]
 [ 1  0  9 28]] 
b:
 [38 20 46 38] 

L:
 [[18  0  0  0]
 [ 0 11  0  0]
 [ 1  4 36  0]
 [ 1  0  9 28]]
U
 [[ 0 -5 -8 -7]
 [ 0  0 -5 -4]
 [ 0  0  0 -5]
 [ 0  0  0  0]]

x = [0.0789576513 0.8448719495 0.7380453396 0.4410118694]
x = [1.3768996916 1.3223387113 1.0313523921 0.9764617421]
x = [0.9056808395 0.9943082792 1.0065215926 1.0012723153]
x = [0.9981877587 0.9965729796 1.0002544096 0.9999829484]
x = [1.0008455103 0.9998905599 0.9999910419 0.9999726826]
x = [1.0000450049 1.0000140055 1.0000009878 0.9999980752]
x = [0.9999964191 1.0000002509 1.0000003389 1.0000000189]
x = [0.9999997723 0.9999998391 1.0000000216 1.0000000012]
x = [1.0000000347 0.9999999898 1.           0.9999999988]


array([1.0000000033, 1.0000000004, 1.          , 0.9999999999])